# Библиотеки

In [7]:
import pandas as pd
import numpy as np
import neptune

from NN import simple_torchpl
from load_data import load
from pl_framework import nn_training
from cv import get_indices
import preprocessing

import torch
import json
import copy

In [8]:
neptune.init('iliaavilov/SIBUR')

Project(iliaavilov/SIBUR)

# Константы

In [9]:
data_path = 'data/'
random_state = 54321
pd.options.display.max_rows = 999
features = ['A_rate', 'A_CH4', 'A_C2H6', 'A_C3H8', 'A_iC4H10', 'A_nC4H10',
            'A_iC5H12', 'A_nC5H12', 'A_C6H14', 'B_rate']

# Загрузка данных

In [10]:
train_features, train_targets, test_features = load(data_path)

In [11]:
cv = get_indices(train_targets, [(pd.to_datetime('2020-03-01 00:00:00'), pd.to_datetime('2020-03-15 00:00:00')),
                                 (pd.to_datetime('2020-03-15 00:00:00'), pd.to_datetime('2020-03-31 00:00:00')),
                                 (pd.to_datetime('2020-04-15 00:00:00'), pd.to_datetime('2020-04-30 00:00:00'))
                                ]
                )

In [12]:
ts = test_features['timestamp'].values
#train_targets = train_targets.drop('timestamp', axis = 'columns')
#test_features = test_features.drop('timestamp', axis = 'columns')
#train_features = train_features.drop('timestamp', axis = 'columns')

# Предсказания для теста

In [13]:
exp_index = 'SIB-294'
n_trial = 6398


exp = neptune.project.get_experiments(exp_index)[0]

channel_names = ['run_parameters']
channels_data = {}
channels_by_name = exp.get_channels()
for channel_name in channel_names:
    channel_id = channels_by_name[channel_name].id

    channels_data[channel_name] = pd.read_csv(
        exp._backend.get_channel_points_csv(exp, channel_id),
        header=None,
        dtype=str
    )
    
values = exp._backend.get_channel_points_csv(exp, channel_id).getvalue()
data = pd.DataFrame(values.split('\n'))
data = data.iloc[:-1]
data[0] = data[0].apply(lambda x: json.loads('{' + x.split(',{')[1].replace("'", '"')))
data.columns = ['params']
#iterations = exp.get_numeric_channels_values('iterations').drop('x', axis = 'columns')
#data = pd.concat([data, iterations], axis = 'columns')

data_trial = data.iloc[n_trial, ]
params_trial = data_trial['params']
#params_trial['mean_best_iter'] = data_trial['iterations']

In [14]:
params_trial['objective'] = 'fair'
params_trial['n_jobs'] = -1
params_trial['n_estimators'] = 3000
params_trial['random_state'] = random_state
params_trial['n_in'] = 0
params_trial['feature_fraction'] = 1
params_trial['bagging_fraction'] = 1

params_trial['n_in'] = 0
params_trial['n_back_A_rate'] = 23
params_trial['n_back_A_CH4'] = 71
params_trial['n_back_A_C2H6'] = 196
params_trial['n_back_A_C3H8'] = 20
params_trial['n_back_A_iC4H10'] = 104
params_trial['n_back_A_iC5H12'] = 194
params_trial['n_back_A_nC4H10'] = 235
params_trial['n_back_A_nC5H12'] = 185
params_trial['n_back_A_C6H14'] = 99
params_trial['n_back_B_rate'] = 116

In [15]:
all_features = train_features.append(test_features, ignore_index = True)
n_backs = []
for feature in ['A_rate', 'A_CH4', 'A_C2H6', 'A_C3H8', 'A_iC4H10',
   'A_nC4H10', 'A_iC5H12', 'A_nC5H12', 'A_C6H14', 'B_rate']:
    n_backs.append(params_trial['n_back_'+feature])

In [16]:
all_features = train_features.append(test_features, ignore_index = True)

X, y_train, cv, params = preprocessing.preprocessing(all_features.copy(), 
                                                     train_targets.copy(), 
                                                     copy.deepcopy(cv), 
                                                     copy.deepcopy(params_trial), 
                                                     pop = False)

current params: {'num_leaves': 2, 'learning_rate': 0.3094159562136855, 'min_child_samples': 2, 'objective': 'fair', 'n_jobs': -1, 'n_estimators': 3000, 'random_state': 54321, 'n_in': 0, 'feature_fraction': 1, 'bagging_fraction': 1, 'n_back_A_rate': 23, 'n_back_A_CH4': 71, 'n_back_A_C2H6': 196, 'n_back_A_C3H8': 20, 'n_back_A_iC4H10': 104, 'n_back_A_iC5H12': 194, 'n_back_A_nC4H10': 235, 'n_back_A_nC5H12': 185, 'n_back_A_C6H14': 99, 'n_back_B_rate': 116}
X shape before dropped nans (7631, 11)
cv before trans: 719
Initial shape of X after dropping nans: (7396, 11)
Initial shape of y: (3647, 5)
cv after trans: 484
shape of X after trans: (7396, 10)
shape of y after trans: (3412,)


In [17]:
X_train = X[:(train_features.shape[0] - max(n_backs))]
X_test = X[(train_features.shape[0] -  max(n_backs)):]

In [18]:
print(y_train.shape)
print(X_train.shape)

(3412,)
(3412, 10)


In [19]:
print(test_features.shape)
print(X_test.shape)

(3984, 11)
(3984, 10)


In [20]:
fold = cv[0]

In [21]:
import pandas as pd
import neptune
import optuna
import neptunecontrib.monitoring.optuna as optuna_utils
import numpy as np
import copy
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
import preprocessing
import os


In [22]:
submission = pd.DataFrame(columns = ['B_C2H6', 'B_C3H8', 'B_iC4H10', 'B_nC4H10'])
submission['timestamp'] = ts
X_tr = X_train.iloc[cv[-1][0], :]
y_tr = y_train.iloc[cv[-1][0]]
X_te = X_train.iloc[cv[-1][1], :]
y_te = y_train.iloc[cv[-1][1]]

train_data = lgb.Dataset(X_tr,
                                 y_tr)
test_data = lgb.Dataset(X_te,
                        y_te
                        )
evals_result = {}

def lgb_scoring(y_hat, data):
    y_true = data.get_label()
    return 'loss', np.mean(np.abs((y_true - y_hat)/y_true)), False
    
test_model = lgb.train(params = params,
                           train_set = train_data,
                           valid_sets=[test_data],
                           valid_names=['test_data'],
                           feval = lgb_scoring,
                           evals_result = evals_result,
                           verbose_eval = False)

target = 'B_C2H6'
predictions = test_model.predict(X_test)
submission[target] = predictions
    
submission.to_csv('submission_C2H6.csv', index = False)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000045 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1620
[LightGBM] [Info] Number of data points in the train set: 484, number of used features: 10
[LightGBM] [Info] Start training from score 4.323842


In [23]:
sub_lgbm = pd.read_csv('submission_C2H6.csv')
sub_nn = pd.read_csv('submission.csv')
sub_nn[target] = sub_lgbm[target]
sub_nn.to_csv('submission_final.csv', index = False)

In [24]:
pd.read_csv('submission_C2H6.csv')

,B_C2H6,B_C3H8,B_iC4H10,B_nC4H10,timestamp
0,4.619516,NaN,NaN,NaN,2020-05-01 00:00:00
1,4.490723,NaN,NaN,NaN,2020-05-01 00:30:00
2,4.642791,NaN,NaN,NaN,2020-05-01 01:00:00
3,4.637100,NaN,NaN,NaN,2020-05-01 01:30:00
4,4.822347,NaN,NaN,NaN,2020-05-01 02:00:00
...,...,...,...,...,...
3979,4.165304,NaN,NaN,NaN,2020-07-22 21:30:00
3980,4.038105,NaN,NaN,NaN,2020-07-22 22:00:00
3981,3.918925,NaN,NaN,NaN,2020-07-22 22:30:00
3982,3.754172,NaN,NaN,NaN,2020-07-22 23:00:00


In [26]:
pd.read_csv('submission_final.csv')

,B_C2H6,B_C3H8,B_iC4H10,B_nC4H10,timestamp
0,4.365124,37.603985,16.775425,23.656387,2020-05-01 00:00:00
1,4.365346,37.604095,16.775760,23.656445,2020-05-01 00:30:00
2,4.365437,37.604200,16.776150,23.656548,2020-05-01 01:00:00
3,4.365876,37.604263,16.776620,23.656752,2020-05-01 01:30:00
4,4.358757,37.604330,16.792437,23.656826,2020-05-01 02:00:00
...,...,...,...,...,...
3979,4.312370,37.627583,16.820438,23.667574,2020-07-22 21:30:00
3980,4.313179,37.627583,16.820396,23.690147,2020-07-22 22:00:00
3981,4.315136,37.627560,16.820278,23.690086,2020-07-22 22:30:00
3982,4.315435,37.627540,16.820276,23.689964,2020-07-22 23:00:00


In [27]:
neptune.project.get_experiments(exp_index)[0].log_metric('leaderboard_mape', 2.6057)